In [ ]:
#use pyspark

In [1]:
import pyspark
from pyspark import SparkContext as sc
from pyspark import SparkConf
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
conf=SparkConf().setAppName("627Project").setMaster("local[*]")
sc=sc.getOrCreate(conf)

In [2]:
# get training and test data
train_rdd = sc.textFile('trainList.txt')
test_rdd = sc.textFile('testList.txt')
scores = train_rdd.map(lambda l: l.split('|')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [3]:
test_data = test_rdd.map(lambda l: l.split('|')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [4]:
rank = 15
numIterations = 10
model = ALS.train(scores, rank, numIterations)

In [5]:
testdata = test_data.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
predictList = predictions.collect()
predictions.count()

119974

In [6]:
testdata.count()

120000

In [7]:
# data processing predict result to    username_item|score
predictScore = []
for record in predictList:
    line = str(record[0][0]) + '|' + str(record[0][1]) + '|' + str(record[1])
    predictScore.append(line)
predictScore.sort()
predictScore[4841]

'201763|76429|80.8042477339065'

In [9]:
predictScore[4847]

'201766|170666|386.65755728653886'

In [10]:
# output predict result to .txt file
predict_txt = []
for record in predictScore:
    record = record + '\n'
    predict_txt.append(record)
f = open('predictScoreRank15.txt', 'a')
f.writelines(predict_txt)
f.close()